# MNIST Dataset from Scratch

attempting to build a neural network to classify handwritten digits from scratch (just numpy + math). 

here's the video that inspiried this: https://www.youtube.com/watch?v=w8yWXqWQYmU

i'll do my best to actually think through everything, and not just copy the video (we'll see about that lol). i want to fully understand the math behind the neural network, which will help me grasp the fundamentals of machine/deep learning. 

i lied btw. 

In [213]:
# importing libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# these are the only libraries/frameworks that i'll use (no tensorflow or pytorch)

# Math 

here i want to write out + explain the math being used to build the neural network. doing this so i actually understand the underlying operations of the neural network, and so that i know what i'm doing (i don't 😘). 

i'll be using the same architecture as the video. the input layer will have 784 (28x28) neurons, each corresponding to a pixel of each training example. the following hidden and output layers each have 10 neurons, with the output layer being a vector of probabilities, enabling the classification of the numbers in the dataset. below, i'll try to map out each step. 

## Forward Propagation

### Input Layer

the input layer looks like this:

$$A^{(0)} = X (784 * m)$$

where m is the number of training examples. 

X represents all of the features (the pixels of each training example). 

### Hidden Layer

the unactivated hidden layer, composed of 10 neurons, is represented as this:

$$Z^{(1)} = W^{(1)} \cdot A^{(0)} + b^{(1)}$$

the dot in the equation represents the dot product of the weights (W1) and the input layer (A0). the bias is later added to the dot product (b1)

### Activation Function (ReLU)
 
without the activation function, the neural network ends up being just a fancy linear equation. the activation function introduces nonlinearity into the model, which is crucial for this use case (classification). 

in this model, i'll be using the ReLU function, like the video:

$$A^{(1)} = g(Z^{(1)}) = ReLU(Z^{(1)})$$

where:

\begin{align}
    ReLU = \begin{cases}
            x, \text{ if x > 0}\\
            0, \text{ if x ≤ 0}
           \end{cases}
\end{align}



the unactivated layer is passed onto the activation function, forming the hidden layer. 

### Output Layer

the unactivated final output layer, like the hidden layer, has 10 neurons, each corresponding to a digit. 

$$Z^{(2)} = W^{(2)} \cdot A^{(1)} + b^{(2)}$$

### Activation Function (Softmax)

the softmax activation function is then applied to the final layer to create a vector of probabilities, which is used for classification. 

the function can be represented as this:

$$A^{(2)} = softmax(Z^{(2)})$$

where:

$$softmax(Z^{(2)}) = \frac {e^{z_i}} {\sum\limits_{j=1}^K e^{z_j}}$$

each value in the resulting vector will be between 0 and 1.

yay, we did it!

except we didn't, because we still have **backpropagation**. yay...

## Backpropagation 

from forward propagation, we obtained a vector of propabilities. using this, we can perform backpropagation, seeing how much the prediction deviated from the actual label of the training example and how much each of the previous weights contributed to the error. 

essentially, we're doing reverse forward propagation (in a way). 

### Output Layer

we first measure the error of the output layer by seeing the difference between the prediction and the label:

$$dZ^{(2)} = A^{(2)} - Y$$

for the label (Y), we will perform one-hot encoding to convert the categories into a binary format (0 and 1) that the predictions can be compared to. 

with this, we can now find the derivatives of the loss functions with respect to the weights and bias in the output layer:

$$dW^{(2)} = \frac {1} {m} dZ^{(2)} A^{(1)T}$$
$$db^{(2)} = \frac {1} {m} \sum dZ^{(2)}$$

### Hidden Layer

we perform similar calcuations for this layer: 

$$dZ^{(1)} = W^{(2)T}dZ^{(2)} * g'(Z^{(2)})$$ 

the g' is the derivative of the activation function ReLU. 

$$dW^{(1)} = \frac {1} {m} dZ^{(1)} X^{T}$$
$$db^{(1)} = \frac {1} {m} \sum dZ^{(1)}$$

### Adjustments

now after ALL of this, we adjust the weights and bias accordingly. 

$$w^{(1)} = w^{(1)} - αdW^{(1)}$$
$$b^{(1)} = b^{(1)} - αdb^{(1)}$$
$$w^{(2)} = w^{(2)} - αdW^{(2)}$$
$$b^{(2)} = b^{(2)} - αdb^{(2)}$$

where α is the learning rate. 

and then we do this over and over again until something cool happens. 

# Loading the Dataset

ok first, i need to actually load + format the data in a way that is compatible for the network. 

In [214]:
# loading the data

dataframe = pd.read_csv("/kaggle/input/digit-recognizer/train.csv") # since the data is a csv file, i can load it in with pandas.read_csv() function.
dataframe.head() # printing the first 5 training examples

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
# now i need to separate the features and targets for classification. 

# dataframe to numpy array
data = np.array(dataframe)
np.random.shuffle(data) # shuffling the data

# i'll also get the shape of the data for variables. 

m, n = data.shape

# separating the features (X) and targets (y) 
data_dev = data[0:1000].T
X = data_dev[1:n]
Y = data[0]

data_train = data[1000:4000].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

# the labels happen to be the first column, rather than the last. 

X, Y # testing, and it worked!

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,  33,  86, 255, 254, 184,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0, 106, 253, 253, 253, 226,
         65,   0,   0,   0,   0,   0,   0,   0,   0,   0,

# Model

now it's time to implement the math (yay...). 

In [216]:
# first, i'm going to initialize the parameters (w and b)

def init_params(): 
    W1 = np.random.rand(10, 784) * 0.01 # in the parantheses, i'm putting the dimensions of the array. 
    B1 = np.random.rand(10, 1)
    W2 = np.random.rand(10, 10) * 0.01
    B2 = np.random.rand(10, 1)

    return W1, B1, W2, B2

In [217]:
# here i'm going to write the ReLU and softmax activation functions. 

def relu(x): 
    # return 0 if x <= 0 else x (first one)
    return np.maximum(0, x) # pretty much the same

def softmax(Z):
    # Z -= np.max(Z, axis=0)  # Subtract max value for numerical stability
    A = np.exp(Z) / np.sum(np.exp(Z), axis=0)
    return A

In [218]:
# now, it's time to write forward prop! i'll see if i can do it without the video. 

def forward_prop(W1, B1, W2, B2, X): 
    Z1 = np.dot(W1, X) + B1 # unactivated hidden layer

    A1 = relu(Z1) # activated hidden layer
    
    Z2 = np.dot(W2, A1) + B2 # unactivated output layer

    A2 = softmax(Z2)

    return Z1, A1, Z1, A2
# aaaaand that's forward prop! now backprop...

In [219]:
# one-hot encoding

def one_hot(x):
    onehot_x = np.zeros((x.size, x.max() + 1))
    onehot_x[np.arange(x.size), x] = 1
    onehot_x = onehot_x.T
    return onehot_x

# i'll be honest, i have no idea what this means... copied ✌️

In [220]:
# derivative of ReLU (copied idc idc)

def d_relu(x): 
    return x > 0
# when booleans are converted to numbers, true = 1 and false = 0 (what he said idk idc) 

In [221]:
# here i'll implement backprop. 

def back_prop(W1, W2, A1, A2, Z1, Z2, X, Y): # Y will be one-hot encoded
    y = one_hot(Y)
    
     # output layer
    dZ2 = 2*(A2 - y)

    dW2 = (1/m) * np.dot(dZ2, A1.T) # weights
    dB2 = 1 / m * np.sum(dZ2, axis=1).reshape(-1, 1)

    # hidden layer
    dZ1 = np.dot(W2.T, dZ2) * d_relu(Z1)
    
    dW1 = (1/m) * np.dot(dZ1, X.T) # weights
    dB1 = 1 / m * np.sum(dZ1, axis=1).reshape(-1, 1)

    return dW1, dB1, dW2, dB2
# yay... (i'm tired)

In [222]:
# separating the update of params bc he did

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2   
    return W1, b1, W2, b2

In [223]:
# accuracy + predictions (copied)

def get_pred(A2):
    return np.argmax(A2, 0)

def get_accuracy(pred, Y):
    print(pred, Y)
    return np.sum(pred == Y) / Y.size

In [224]:
# now we can perform gradient descent 

def gradient_descent(X, Y, iters, alpha):
    W1, B1, W2, B2 = init_params()
    for i in range(iters): 
        Z1, A1, Z2, A2 = forward_prop(W1, B1, W2, B2, X)   
        dW1, dB1, dW2, dB2 = back_prop(W1, W2, A1, A2, Z1, Z2, X, Y)
        W1, B1, W2, B2 = update_params(W1, B1, W2, B2, dW1, dB1, dW2, dB2, alpha)

        if (i%10) == 0: 
            print("Iteration: ", i)
            pred = get_pred(A2)
            print("Accuracy: ", get_accuracy(pred, Y))
    return W1, B1, W2, B2           

In [225]:
# now time to apply it!

W1, B1, W2, B2 = gradient_descent(X_train, Y_train, 1000, 1)
 
print(W1, B1, W2, B2) 

Iteration:  0
[1 1 1 ... 1 1 1] [5 2 8 ... 1 3 7]
Accuracy:  0.11
Iteration:  10
[3 3 3 ... 2 3 3] [5 2 8 ... 1 3 7]
Accuracy:  0.10733333333333334
Iteration:  20
[3 3 3 ... 3 3 3] [5 2 8 ... 1 3 7]
Accuracy:  0.107
Iteration:  30
[3 3 3 ... 3 3 3] [5 2 8 ... 1 3 7]
Accuracy:  0.15733333333333333
Iteration:  40
[0 0 3 ... 1 3 3] [5 2 8 ... 1 3 7]
Accuracy:  0.25
Iteration:  50
[0 0 3 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.3443333333333333
Iteration:  60
[0 0 3 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.384
Iteration:  70
[2 0 3 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.4736666666666667
Iteration:  80
[2 2 3 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.5853333333333334
Iteration:  90
[2 2 3 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.6516666666666666
Iteration:  100
[2 2 8 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.698
Iteration:  110
[2 2 8 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.729
Iteration:  120
[2 2 8 ... 1 3 7] [5 2 8 ... 1 3 7]
Accuracy:  0.7586666666666667
Iteration:  130
[2 2 8 

after hours of debugging + not knowing what i'm doing, i achieved ~95% accuracy with 1000 iterations and a learning rate of 1 (??????). i have no idea what i did, but i'm tired. 

i can now say that i made a nn from scratch (with help.... don't tell anyone).

ok bye sleep time now. 